In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 7.3MB/s 
     |████████████████████████████████| 1.1MB 15.0MB/s 
     |████████████████████████████████| 194kB 28.3MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 61kB 10.9MB/s 
Mounted at /content/gdrive


In [2]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [3]:
path = untar_data(URLs.PETS)

In [4]:
pet = DataBlock(blocks=(ImageBlock, CategoryBlock),
                get_items=get_image_files,
                splitter=RandomSplitter(),
                get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'),
                item_tfms=Resize(460),
                batch_tfms=aug_transforms(mult=2)
                )

In [5]:
dls = pet.dataloaders(path/"images")

In [6]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


epoch,train_loss,valid_loss,error_rate,time
0,1.928358,0.400358,0.140731,02:30


epoch,train_loss,valid_loss,error_rate,time
0,0.562459,0.231448,0.072395,03:33


In [7]:
learn.export()

In [8]:
dd = Path()
dd.ls(file_exts='.pkl')

(#1) [Path('export.pkl')]

In [9]:
learn_inf = load_learner(dd/'export.pkl')

In [10]:
out_pl = widgets.Output()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')
btn_run

Button(description='Classify', style=ButtonStyle())

In [14]:
def on_click_classify(change):
  img = PILImage.create(btn_upload.data[-1])
  out_pl.clear_output()
  with out_pl: display(img.to_thumb(128,128))
  pred,pred_idx,probs = learn_inf.predict(img)
  lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.6f}'
btn_run.on_click(on_click_classify)

In [15]:
btn_upload = widgets.FileUpload()

In [18]:
VBox([widgets.Label('Select your Pet!'),
      btn_upload, btn_run, out_pl, lbl_pred])